In [1]:
from fitting import fit_all_models_parallel, fit_model_sequentially
from agents.agents_configs import all_models, get_models_for_fitting, toggle_model_inclusion

# Example usage:
# Get all currently included models
# models_to_fit = get_models_for_fitting()

# Get only RW models (exclude RLWM)
# models_to_fit = get_models_for_fitting(include_names=['rw'], exclude_names=['rlwm'])

# Get models with specific patterns
# models_to_fit = get_models_for_fitting(include_names=['rw_noise'])

# Turn off specific models
# toggle_model_inclusion(['rw_noise_bias_pav_dynamic_collins_decay_both'], include=False)

# For standard progression, do:
# toggle_model_inclusion(['rw_noise_bias_pav_dynamic_collins_decay_both', 'rw_noise_bias_pav_dynamic_collins_decay_q'], include=False)
# models_to_fit = get_models_for_fitting(include_names=['rw'], exclude_names=['rlwm'])

In [2]:
def print_models(cfgs):
    print("Current models available:")
    print("________________________")
    for model in cfgs:
        print(f"  - {model.name}")

In [3]:
# PyBADS Fitting Pipeline

# 1. Select models for fitting
toggle_model_inclusion(['rw_noise_bias_pav_dynamic_collins_decay_both', 
                       'rw_noise_bias_pav_dynamic_collins_decay_q'], include=False)

models_to_fit = get_models_for_fitting(include_names=['rw'], exclude_names=['rlwm'])

print_models(models_to_fit)

Current models available:
________________________
  - rw
  - rw_noise
  - rw_noise_bias
  - rw_noise_bias_pav_dynamic
  - rw_noise_bias_asym_reward
  - rw_noise_bias_pav_dynamic_asym_decay_both
  - rw_bayesian_control
  - rw_bayesian_control_counterfactual
  - rw_bayesian_pav
  - rw_bayesian_pav_counterfactual
  - rw_omega_control
  - rw_reward_rate_bias
  - rw_context_decay
  - rw_win_stay_boost


In [5]:
# 2. Set up data and fitting parameters for PyBADS
from agents.robot_dataset import RobotDataset
data = RobotDataset()
toolbox = "pybads"
fit_with_slurm = True

# PyBADS-specific options
bads_options = {
    'max_fun_evals': 1000,
    'tol_mesh': 1e-8,
    'tol_fun': 1e-6,
    'tol_stall_iters': 20,
}

✓ Reward recoding successful: 10240 trials with rewards [np.int64(-1), np.int64(1)]
  Reward distribution: +1=6813, -1=3427


In [6]:
# 3a) Fit all selected models with PyBADS
pybads_results = fit_all_models_parallel(
        models_to_fit=models_to_fit,
        data=data,
        toolbox="pybads",
        fit_with_slurm=True,
        n_starts = 1,
        bads_options=None,
        developmental_subjects=['sub-001', 'sub-020'],
        vbmc_init_with_bads=False
    )

✓ Reward recoding successful: 10240 trials with rewards [np.int64(-1), np.int64(1)]
  Reward distribution: +1=6813, -1=3427
Submitting jobs for 6 models with 2 subjects each...


Submitting jobs:   0%|          | 0/12 [00:00<?, ?it/s]

Using PYBADS for optimization
Using PYBADS for optimization
Using PYBADS for optimization
Using PYBADS for optimization
Using PYBADS for optimization
Using PYBADS for optimization
Submitted 12 total jobs to SLURM



Computing population statistics for each model...
Using PYBADS for optimization
✓ rw: 2/2 subjects successful
Using PYBADS for optimization
✓ rw_noise: 2/2 subjects successful
Using PYBADS for optimization
✓ rw_noise_bias: 2/2 subjects successful
Using PYBADS for optimization
✓ rw_noise_bias_pav_dynamic: 2/2 subjects successful
Using PYBADS for optimization
✓ rw_noise_bias_asym_reward: 2/2 subjects successful
Using PYBADS for optimization
✓ rw_noise_bias_pav_dynamic_asym_decay_both: 2/2 subjects successful

All results saved to: model_fits/logs/parallel_fitting/20250626_153351/all_models_results.pkl
Fitted 6 models successfully!


In [7]:
# 3b) Fit all selected models with PyVBMC (using PyBADS for initialization)

# PyVBMC-specific options
vbmc_options = {
    'max_fun_evals': 500,      # Fewer evaluations than PyBADS since it's more efficient
    'tol_con_loss': 0.01,      # Convergence tolerance
    'fun_eval_start': 10,      # Function evaluations before starting GP
    'k_warmup': 5,             # Warmup iterations
    'max_iter': 100            # Maximum iterations
}

# PyBADS initialization options (for PyVBMC)
vbmc_bads_init_options = {
    'max_fun_evals': 100,      # Quick initialization with PyBADS
    'tol_fun': 1e-3,           # Looser tolerance for initialization
    'tol_mesh': 1e-5           # Mesh tolerance
}

# Fit with PyVBMC using PyBADS initialization
pyvbmc_results = fit_all_models_parallel(
        models_to_fit=models_to_fit,
        data=data,
        toolbox="pyvbmc",              # Use PyVBMC
        fit_with_slurm=True,
        n_starts=1,                    # Fewer starts needed with VBMC
        vbmc_options=vbmc_options,
        vbmc_bads_init_options=vbmc_bads_init_options,
        developmental_subjects=['sub-001', 'sub-020'],
        vbmc_init_with_bads=True       # Enable PyBADS initialization
    )

print("✓ PyVBMC fitting completed with PyBADS initialization!")

✓ Reward recoding successful: 10240 trials with rewards [np.int64(-1), np.int64(1)]
  Reward distribution: +1=6813, -1=3427
Submitting jobs for 6 models with 2 subjects each...


Submitting jobs:   0%|          | 0/12 [00:00<?, ?it/s]

Using PYVBMC for optimization
Will initialize VBMC with PyBADS
Using PYVBMC for optimization
Will initialize VBMC with PyBADS
Using PYVBMC for optimization
Will initialize VBMC with PyBADS
Using PYVBMC for optimization
Will initialize VBMC with PyBADS
Using PYVBMC for optimization
Will initialize VBMC with PyBADS
Using PYVBMC for optimization
Will initialize VBMC with PyBADS
Submitted 12 total jobs to SLURM



Computing population statistics for each model...
Using PYVBMC for optimization
Will initialize VBMC with PyBADS
✓ rw: 2/2 subjects successful
Using PYVBMC for optimization
Will initialize VBMC with PyBADS
✓ rw_noise: 2/2 subjects successful
Using PYVBMC for optimization
Will initialize VBMC with PyBADS
✓ rw_noise_bias: 2/2 subjects successful
Using PYVBMC for optimization
Will initialize VBMC with PyBADS
✓ rw_noise_bias_pav_dynamic: 2/2 subjects successful
Using PYVBMC for optimization
Will initialize VBMC with PyBADS
✓ rw_noise_bias_asym_reward: 2/2 subjects successful
Using PYVBMC for optimization
Will initialize VBMC with PyBADS
✓ rw_noise_bias_pav_dynamic_asym_decay_both: 2/2 subjects successful

All results saved to: model_fits/logs/parallel_fitting/20250626_153457/all_models_results.pkl
Fitted 6 models successfully!
✓ PyVBMC fitting completed with PyBADS initialization!


In [7]:
# 4. Compare PyBADS vs PyVBMC Results

def compare_fitting_results(pybads_results, pyvbmc_results):
    """Compare fitting results between PyBADS and PyVBMC"""
    print("=" * 60)
    print("FITTING RESULTS COMPARISON: PyBADS vs PyVBMC")
    print("=" * 60)
    
    for model_name in pybads_results.keys():
        if model_name in pyvbmc_results:
            print(f"\nModel: {model_name}")
            print("-" * 40)
            
            # PyBADS results
            bads_pop_stats = pybads_results[model_name]['population_statistics']
            bads_n_successful = pybads_results[model_name]['n_successful']
            bads_n_total = pybads_results[model_name]['n_subjects']
            
            # PyVBMC results  
            vbmc_pop_stats = pyvbmc_results[model_name]['population_statistics']
            vbmc_n_successful = pyvbmc_results[model_name]['n_successful']
            vbmc_n_total = pyvbmc_results[model_name]['n_subjects']
            
            print(f"Convergence rate:")
            print(f"  PyBADS:  {bads_n_successful}/{bads_n_total} ({100*bads_n_successful/bads_n_total:.1f}%)")
            print(f"  PyVBMC:  {vbmc_n_successful}/{vbmc_n_total} ({100*vbmc_n_successful/vbmc_n_total:.1f}%)")
            
            # Compare parameter estimates
            if 'parameters' in bads_pop_stats and 'parameters' in vbmc_pop_stats:
                print(f"Parameter estimates:")
                for param in bads_pop_stats['parameters'].keys():
                    if param in vbmc_pop_stats['parameters']:
                        bads_mean = bads_pop_stats['parameters'][param]['mean']
                        vbmc_mean = vbmc_pop_stats['parameters'][param]['mean']
                        print(f"  {param:12s}: PyBADS={bads_mean:.3f}, PyVBMC={vbmc_mean:.3f}")
            
            # Compare log-likelihood
            if 'log_likelihood' in bads_pop_stats and 'log_likelihood' in vbmc_pop_stats:
                bads_ll = bads_pop_stats['log_likelihood']['total']
                vbmc_ll = vbmc_pop_stats['log_likelihood']['total']
                print(f"Total Log-Likelihood:")
                print(f"  PyBADS:  {bads_ll:.1f}")
                print(f"  PyVBMC:  {vbmc_ll:.1f}")
                print(f"  Difference: {vbmc_ll - bads_ll:.1f}")
                
            # Model evidence (only available for PyVBMC)
            if 'model_evidence' in vbmc_pop_stats and vbmc_pop_stats['model_evidence']:
                evidence = vbmc_pop_stats['model_evidence'].get('total', 'N/A')
                print(f"Model Evidence (PyVBMC only): {evidence}")

# Run the comparison
compare_fitting_results(pybads_results, pyvbmc_results)

NameError: name 'pybads_results' is not defined

In [5]:
# Test loading functions
from fitting import find_latest_parallel_fit_results, list_all_parallel_fit_results

# List all available results
print("=== Testing Loading Functions ===")
all_results = list_all_parallel_fit_results()

# Try to load the latest results
latest_results, latest_path = find_latest_parallel_fit_results()

if latest_results:
    print(f"\n✓ Successfully loaded {len(latest_results)} models from: {latest_path}")
    print("Models found:", list(latest_results.keys()))
    
    # Check structure of one model
    first_model = list(latest_results.keys())[0]
    model_data = latest_results[first_model]
    print(f"\nStructure check for '{first_model}':")
    print(f"  - fit_toolbox: {model_data.get('fit_toolbox', 'N/A')}")
    print(f"  - n_subjects: {model_data.get('n_subjects', 'N/A')}")
    print(f"  - n_successful: {model_data.get('n_successful', 'N/A')}")
    print(f"  - has population_statistics: {'population_statistics' in model_data}")
    print(f"  - has individual_results: {'individual_results' in model_data}")
else:
    print("✗ Failed to load results")

=== Testing Loading Functions ===
Found 17 parallel fitting directories:
  1. 20250626_153457 - ✓ Has results
  2. 20250626_153351 - ✓ Has results
  3. 20250626_134248 - ✓ Has results
  4. 20250626_134219 - ✗ No results file
  5. 20250626_134005 - ✓ Has results
  6. 20250626_133944 - ✗ No results file
  7. 20250626_122734 - ✓ Has results
  8. 20250626_122106 - ✓ Has results
  9. 20250626_111358 - ✓ Has results
  10. 20250626_110206 - ✓ Has results
  11. 20250626_110126 - ✗ No results file
  12. 20250626_102621 - ✓ Has results
  13. 20250626_102258 - ✗ No results file
  14. 20250626_101640 - ✗ No results file
  15. 20250626_101345 - ✗ No results file
  16. 20250626_100821 - ✗ No results file
  17. 20250626_100216 - ✓ Has results
Loaded parallel fit results from: /mnt/z/3017083.01/behavioral_study/scripts/gonogo-simfit/model_fits/logs/parallel_fitting/20250626_153457/all_models_results.pkl (dill)
Found 6 models fitted

✓ Successfully loaded 6 models from: /mnt/z/3017083.01/behavioral_stu